# Segmenting and Clustering Neighborhoods in Toronto - Part 1 (Week 3)

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown on coursera


### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [50]:
import numpy as np 
import json
from geopy.geocoders import Nominatim
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install folium==0.5.0
import folium

In [7]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)


required_cols = ['Postcode', 'Borough','Neighbourhood']


for table in tables:
    if(str(np.array_equal(np.array(table.columns),np.array(required_cols)))=="True"):
        pstl_data_df = pd.DataFrame(table)    
    break
print("The Shape of Dataframe is -", pstl_data_df.shape)
pstl_data_df.head()

The Shape of Dataframe is - (288, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
pstl_data_df = pstl_data_df[pstl_data_df.Borough!="Not assigned"]
print(" The Shape of Dataframe is -", pstl_data_df.shape)
pstl_data_df.head()

 The Shape of Dataframe is - (211, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [9]:
pstl_data_df['new_neighbour'] = np.where(pstl_data_df['Neighbourhood']=='Not assigned',pstl_data_df['Borough'],pstl_data_df['Neighbourhood'])
pstl_data_df.head(10)

,Postcode,Borough,Neighbourhood,new_neighbour
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,Harbourfront,Harbourfront
5,M5A,Downtown Toronto,Regent Park,Regent Park
6,M6A,North York,Lawrence Heights,Lawrence Heights
7,M6A,North York,Lawrence Manor,Lawrence Manor
8,M7A,Queen's Park,Not assigned,Queen's Park
10,M9A,Etobicoke,Islington Avenue,Islington Avenue
11,M1B,Scarborough,Rouge,Rouge
12,M1B,Scarborough,Malvern,Malvern


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [10]:
can_postal_cd_df = pd.DataFrame(pstl_data_df.groupby(['Postcode','Borough'])['new_neighbour'].apply(','.join).reset_index())

can_postal_cd_df = can_postal_cd_df.rename(columns = {"Postcode": "PostalCode","new_neighbour":"Neighborhood"})

In [11]:
print(can_postal_cd_df.head())
print("\n The Final Shape of the dataframe is  -",can_postal_cd_df.shape)

  PostalCode      Borough                          Neighborhood
0        M1B  Scarborough                         Rouge,Malvern
1        M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2        M1E  Scarborough       Guildwood,Morningside,West Hill
3        M1G  Scarborough                                Woburn
4        M1H  Scarborough                             Cedarbrae

 The Final Shape of the dataframe is  - (103, 3)


### We see above the shape of the new data frame created is (103,3) - Ending of Part 1

# PART 2 -  Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [51]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Geo Coder imported!")


Solving environment: done

# All requested packages already installed.

Installation Done!
Geo Coder imported!


Create function to get lat and long data

In [52]:
def get_geocoder(postal_code_from_df):

    lat_lng_coords = None
 
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

Adding latitute and longitude to Pandas Dataframe

In [53]:
can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'] = zip(*can_postal_cd_df['PostalCode'].apply(get_geocoder))
can_postal_cd_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [54]:
print("The Shape of the dataframe is - ",can_postal_cd_df.shape)

The Shape of the dataframe is -  (103, 5)


## Map of Toronto (Using Geolocator)

In [55]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto, Ontario are 43.653963, -79.387207.


## Using Folium to Map

In [56]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(can_postal_cd_df['Latitude'], can_postal_cd_df['Longitude'], can_postal_cd_df['PostalCode'], can_postal_cd_df['Borough'], can_postal_cd_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

End of Part 2